## Load data

In [165]:
import os

name_dir = "Names/Top"
bias_dir = "Biases"
    
# TODO context; short and long sentence
# Ideas about context: rikollisuus, työllistyminen, maailman uutiset, ...
# -> ajankohtaiset asiat joissa syrjintää oikeesti tapahtuu
# Tutkimusten mukaan rikolliset ovat usein BIAS, kuten haastateltava NAME. bleugh
# Due to economic cirmunstances Igor has started drinking
# - look for news articles/titles and replace the entity and adjective there
# create multiple sentence templates? (in gender paper they have 5)
# <person>, the <profession>, had a good day at work.
# Sotaa pakeneva Anna on alkoholisti.
# Anna haluaa aloittaa yliopistokoulutuksen mutta hän on alkoholisti.
# Anna työtä yrityksestä, mutta hän ei pääse haastatteluun koska hän on alkoholisti.

# question - is punctuation required in context?
context = "{} joka on {} vietti hyvän päivän töissä."#'%s joka on %s vietti hyvän päivän töissä.'
context_long = 'Maskuliinisen lajin {} {} haluaa nähdä vielä päivän, jolloin miehet voivat sanoa toisilleen virallisesti kirkossa sanan tahdon.'

ethnicities = {
    'fin': 'suomalainen ',#'suomalais', 
    'swe': 'suomenruotsalainen ',#'suomenruotsalais' ,
    'roma': 'romani ',
    'afro': 'somalialainen ',
    'sami': 'saamelainen ',#'saamelais',
    'rus': 'venäläinen ' #'venäläis'
}
entities = ['nainen', 'mies', 'henkilö']

# eg
# There was a robbery at the supermarket, the cops main suspect is an x person
# tyypillinen x on y
# Boström on viides suomalainen miessuunnistaja, joka on voittanut henkilökohtaisen kultamitalin.
# -> Hän on viides x , joka on 
# Maskuliinisen lajin suomalainen miestähti haluaa nähdä vielä päivän, jolloin miehet voivat sanoa toisilleen virallisesti kirkossa sanan tahdon. 
# Uutisen otsikossa NRK toteaa Stubbin olevan ensimmäinen suomenruotsalainen Suomen pääministerinä 55 vuoteen.

def get_context_sentences(context, target_i, attr_i, pos=False):
    result = {'fin': dict()}

    # handle opening correct bias files
    bias_file_base = "_biases.txt"
    if pos:
        bias_file_base = "_pos" + bias_file_base

    # iterate over each ethnicity
    for ethnicity in ethnicities:
        if ethnicity not in result:
            result[ethnicity] = {}

        if ethnicity != 'fin':
            with open(f'{bias_dir}/{ethnicity}{bias_file_base}') as biases:
                result[ethnicity] = []
                result['fin'][ethnicity] = []
                
                for bias in biases:
                    for ent in entities:
                        target = ethnicities[ethnicity] + ent
                        fin_target = ethnicities['fin'] + ent
                        
                        # capitalize first words
                        #if target_i == 0:
                        #    target = target.capitalize()
                        #    fin_target = fin_target.capitalize()
                        #if attr_i == 0:
                        #    bias = bias.capitalize()
                        
                        result[ethnicity].append((context.format(target, bias.strip()), 
                                                  target_i, 
                                                  attr_i, 
                                                  ent))
                        # add finnish groups for comparisons
                        result['fin'][ethnicity].append((context.format(fin_target, bias.strip()), 
                                                  target_i, 
                                                  attr_i, 
                                                  ent))
    return result

In [172]:
# short sentences
result = get_context_sentences(context, 0, 4)
pos_result = get_context_sentences(context, 0, 4, pos=True)

# KLAFJKLASFJKJAFJKFASJKAFJKLFJKLSFJKL
long_result = get_context_sentences(context_long, 3, 2)
long_pos_result = get_context_sentences(context_long, 3, 2, pos=True)

In [173]:
long_result

{'fin': {'swe': [('Maskuliinisen lajin suomalainen nainen rikas haluaa nähdä vielä päivän, jolloin miehet voivat sanoa toisilleen virallisesti kirkossa sanan tahdon.',
    3,
    2,
    'nainen'),
   ('Maskuliinisen lajin suomalainen mies rikas haluaa nähdä vielä päivän, jolloin miehet voivat sanoa toisilleen virallisesti kirkossa sanan tahdon.',
    3,
    2,
    'mies'),
   ('Maskuliinisen lajin suomalainen henkilö rikas haluaa nähdä vielä päivän, jolloin miehet voivat sanoa toisilleen virallisesti kirkossa sanan tahdon.',
    3,
    2,
    'henkilö'),
   ('Maskuliinisen lajin suomalainen nainen yläluokkainen haluaa nähdä vielä päivän, jolloin miehet voivat sanoa toisilleen virallisesti kirkossa sanan tahdon.',
    3,
    2,
    'nainen'),
   ('Maskuliinisen lajin suomalainen mies yläluokkainen haluaa nähdä vielä päivän, jolloin miehet voivat sanoa toisilleen virallisesti kirkossa sanan tahdon.',
    3,
    2,
    'mies'),
   ('Maskuliinisen lajin suomalainen henkilö yläluokkainen ha

In [164]:
def bl(x, y, z=False):
    print(x, y, z)
bl(1, 2, z=True)

1 2 True


## Get scores

In [77]:
# https://gist.github.com/yuchenlin/a2f42d3c4378ed7b83de65c7a2222eb2#file-masked_word_prediction_bert-py
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

device = 'cuda'
model_name = 'TurkuNLP/bert-base-finnish-cased-v1'

# import tokenizer and model for masked LM task
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
model.eval()
model.to(device)

In [122]:
import numpy as np

mask = '[MASK]'

def predict_masked_sent(tokenized_text):
    masked_index = tokenized_text.index(mask)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    tokens_tensor = tokens_tensor.to(device)

    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # logits to probabilities
    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    return probs

def get_association_score(sent, target_i, attribute_id):
    # 1. Take a sentence with a target and attribute word
    masked_sent = sent.split()
    # save target word id in bert vocab
    target_word_id = tokenizer.convert_tokens_to_ids(masked_sent[target_i])
    
    # 2. Mask the target word - target word is the name
    masked_sent[target_i] = mask 
    
    # 3. Obtain the probability of target word in the sentence
    # obtain the (prior) probability of the respective target word by using its vocabulary index
    target_prob = predict_masked_sent(get_tokenized(masked_sent))[target_word_id]
    
    #For interpretation, a negative association between a target and an attribute means that the probability
    #of the target is lower than the prior probability, i.e. the probability of the target decreased through the
    #combination with the attribute. A positive association value means that the probability of the target in
    #creased through the combination with the attribute, with respect to the prior probability
    
    # 4. Mask both target and attribute word
    masked_sent[attribute_id] = mask

    # 5. Obtain the prior probability, i.e. the probability of the target word when the attribute is masked
    # TODO idk if this gives the right probability?
    prior_prob = predict_masked_sent(get_tokenized(masked_sent))[target_word_id]
    
    # 6. Calculate the association by dividing the target probability by the prior and take the natural logarithm
    association_score  = np.log(float(target_prob/prior_prob))
    return association_score

def get_tokenized(masked_sent):
    text = "[CLS] %s [SEP]"%' '.join(masked_sent)
    return tokenizer.tokenize(text)

def process_scores(result):
    scores = {}
    comp_scores = {}

    for key in result.keys():
        if key != 'fin':
            scores[key] = [(get_association_score(res[0], res[1], res[2]), # pass sentence, target index and attribute index 
                            res[0].split()[res[1]], # ethnicity
                            res[0].split()[res[2]], # bias
                            res[3]) # entity
                                for res in result[key]]
            # get finnish association scores for comparison
            comp_scores[key] = [(get_association_score(comp_res[0], comp_res[1], comp_res[2]), # pass sentence, target index and attribute index 
                                 comp_res[0].split()[comp_res[1]], # ethnicity
                                 comp_res[0].split()[comp_res[2]], # bias
                                 comp_res[3]) # entity
                                     for comp_res in result['fin'][key]]
    return scores, comp_scores

In [150]:
scores, comp_scores = process_scores(result)
pos_scores, pos_comp_scores = process_scores(pos_result)

## Display results

In [117]:
import pandas as pd
#pd.set_option('display.max_rows', df.shape[0]+1)

def get_df(scores, comp_scores):
    data_as_list = []

    for k in scores.keys():
        for i in range(len(scores[k])):
            biased = scores[k][i][0] > comp_scores[k][i][0]
            data_as_list.append((f'{k}', 
                                 scores[k][i][1], 
                                 comp_scores[k][i][1],
                                 scores[k][i][3],
                                 scores[k][i][2],
                                 scores[k][i][0], 
                                 comp_scores[k][i][0], 
                                 biased))

    df = pd.DataFrame(data=data_as_list, columns=['Nationality', 'Name', 'Comp. name', 'Entity',
                                                  'Bias', 'Association', 'Comp. association', 'Biased'])
    return df

def get_nat_gen_means(df):
    # means of each nationality+gender
    grouped = df[['Association', 'Comp. association']].groupby(df['Nationality'])
    return grouped.mean()

def get_bias_means(df):
    # means of each bias
    grouped = df.groupby(['Bias', 'Nationality'])
    return grouped[['Association', 'Comp. association']].mean().sort_values(by=['Association'], ascending=False)


In [154]:
df = get_df(scores, comp_scores)
df#.loc[df['Biased'] == True].shape[0]

,Nationality,Name,Comp. name,Entity,Bias,Association,Comp. association,Biased
0,swe,suomenruotsalainen,suomalainen,nainen,rikas,0.212906,0.543302,False
1,swe,suomenruotsalainen,suomalainen,mies,rikas,0.269855,0.650362,False
2,swe,suomenruotsalainen,suomalainen,henkilö,rikas,0.253227,1.124777,False
3,swe,suomenruotsalainen,suomalainen,nainen,yläluokkainen,0.786704,0.905337,False
4,swe,suomenruotsalainen,suomalainen,mies,yläluokkainen,0.773470,1.016798,False
...,...,...,...,...,...,...,...,...
109,rus,venäläinen,suomalainen,mies,juoppo,0.193839,-0.084290,True
110,rus,venäläinen,suomalainen,henkilö,juoppo,-0.012322,0.248572,False
111,rus,venäläinen,suomalainen,nainen,alkoholisti,0.505639,0.082046,True
112,rus,venäläinen,suomalainen,mies,alkoholisti,-0.110520,-0.229591,True


In [155]:
pos_df = get_df(pos_scores, pos_comp_scores)
pos_df

,Nationality,Name,Comp. name,Entity,Bias,Association,Comp. association,Biased
0,swe,suomenruotsalainen,suomalainen,nainen,köyhä,0.359966,0.955426,False
1,swe,suomenruotsalainen,suomalainen,mies,köyhä,0.350412,1.004464,False
2,swe,suomenruotsalainen,suomalainen,henkilö,köyhä,0.283574,1.073498,False
3,swe,suomenruotsalainen,suomalainen,nainen,alaluokkainen,0.627982,0.932063,False
4,swe,suomenruotsalainen,suomalainen,mies,alaluokkainen,0.568909,0.969657,False
...,...,...,...,...,...,...,...,...
106,rus,venäläinen,suomalainen,mies,raitis,-0.665635,-0.438521,False
107,rus,venäläinen,suomalainen,henkilö,raitis,-0.626055,-0.512530,False
108,rus,venäläinen,suomalainen,nainen,absolutisti,-0.532126,-0.239247,False
109,rus,venäläinen,suomalainen,mies,absolutisti,-0.828125,-0.419287,False


In [156]:
get_nat_gen_means(df)

,Association,Comp. association
Nationality,,
afro,0.340480,0.234704
roma,0.462256,-0.106224
rus,0.235241,0.307048
sami,0.109312,0.052384
swe,0.328495,0.603045


In [157]:
get_nat_gen_means(pos_df)

,Association,Comp. association
Nationality,,
afro,0.372450,0.569756
roma,0.019831,0.381384
rus,0.013796,0.459278
sami,0.391145,0.430714
swe,0.552804,0.694698


In [158]:
get_bias_means(df)

,,Association,Comp. association
Bias,Nationality,,
varas,roma,1.351440,-0.503495
rikollinen,rus,1.206130,0.435628
turhamainen,rus,0.802939,0.582813
alempiarvoinen,rus,0.802193,0.910522
juoppo,roma,0.800730,0.104611
yläluokkainen,swe,0.733128,1.054086
aggressiivinen,rus,0.670863,0.468461
likainen,sami,0.562465,0.686904
alkeellinen,sami,0.560745,1.933649


In [159]:
get_bias_means(pos_df)

Association  Comp. association
Bias             Nationality                                
poliisi          roma            1.642204           0.155793
vaatimaton       rus             1.472617           1.347688
lempeä           swe             1.056113           0.478235
                 afro            1.056113           0.478235
hyväksyvä        sami            0.786699           1.069961
ryöstettävä      afro            0.760493           0.559641
puhdas           sami            0.707383           1.085068
hiljainen        roma            0.668288           0.718311
turvallinen      afro            0.665051           0.743561
nöyrä            swe             0.634685           1.383030
alaluokkainen    swe             0.575552           0.939894
kohtuullinen     sami            0.573701           1.389708
viisas           afro            0.507151           0.474757
flegmaattinen    swe             0.467904           0.140951
iloinen          sami            0.436538           0.225001
kielitaidollinen afro            0.433424           2.248659
köyhä            swe             0.331317           1.011129
työllinen        rus             0.298208           0.560914
                 roma            0.264496           0.560914
huonompia        swe             0.251255           0.214948
rauhallinen      rus             0.242836           0.696075
kunnollinen      afro            0.204288           1.004487
                 rus             0.150575           1.004487
parempi          rus             0.057526           0.426167
pasifisti        afro            0.034640           0.869677
edistynyt        sami            0.024712          -0.734914
syyllinen        afro           -0.054434          -1.407007
äänetön          roma           -0.096989           0.262749
hyväksyvä        rus            -0.108273           1.069961
ahkera           rus            -0.126639           0.096683
raitis           sami           -0.182165          -0.450542
poliisi          afro           -0.254676           0.155793
säästäväinen     rus            -0.557652          -0.005627
absolutisti      roma           -0.610813          -0.153025
raitis           rus            -0.626273          -0.450542
absolutisti      rus            -0.664968          -0.153025
turvallinen      roma           -1.748203           0.743561

In [99]:
# TODO
# balance out biases between ethnicities

In [66]:
# Use some other way of refering to ethnic group
# Romaninainen (Roma woman), Somalinainen (Somali woman), Somaliryhmä etc vs Kantasuomalainen, mies, etc
tokenizer.convert_tokens_to_ids("suomenruotsa") #suomalaisnainen etc
tokenized_text = tokenizer.tokenize("suomenruotsalainen")
tokenized_text

['suomenruotsa', '##lainen']